In [1]:
import re
import typing
import json
import config
import numpy as np
import pandas as pd
import requests
import tqdm


In [2]:
CFG = config.Config()

In [10]:
DATA_PATH: str = './dataset'
DATA_SUBSET: str = 'full'

LANGUAGE: str = 'English'
TOPIC: str = 'ukraine'
GROUPER: str = 'retrieved_source'
EXTRACTOR: str = r'\d\.\s(.+)\n'

SAMPLE_SIZE: int = 100

MODEL: str = 'llama3.1:70b-instruct-q6_K'  # options: 'gemma:7b-instruct-q6_K', 'gemma2:27b-instruct-q6_K', 'llama3.1:8b-instruct-q6_K', 'llama3.1:70b-instruct-q6_K', 'mistral:7b-instruct-v0.3-q6_K', 'mistral-large:123b-instruct-2407-q6_K', 'mixtral:8x7b-instruct-v0.1-q6_K', 'mixtral:8x22b-instruct-v0.1-q6_K', 'phi3:14b-medium-128k-instruct-q6_K' or 'qwen2:72b-instruct-q6_K'


In [4]:
data: pd.DataFrame = (
    pd.read_parquet(f'{DATA_PATH}.{DATA_SUBSET}.parquet')
    .pipe(lambda _df: _df[_df['language'] == LANGUAGE])
    .pipe(lambda _df: _df[_df['topic'] == TOPIC])
)
data

,persona,model,topic,language,text,retrieved_source,annotation.topic,annotation.persona,annotation.authenticity
id,,,,,,,,,
c0e20a320406ea9273e4b8a5f9d5d6a93e21e110c8bf77ae,alt_right,mixtral:8x7b-instruct-v0.1-q6_K,ukraine,English,Sad to see Ukraine's energy infrastructure und...,https://oilprice.com/Energy/Energy-General/Rus...,NaN,NaN,NaN
695b60f976c5dad0e0852181c0f0e04cce6ec93401b91a9a,neutral,mixtral:8x7b-instruct-v0.1-q6_K,ukraine,English,China's Xi proposes 4 priorities for Ukraine. ...,https://www.cnbc.com/2024/04/16/ukraine-war-li...,NaN,NaN,NaN
7377850436bd8d2889e0f2037e2ecc379c9a5b821d60e5ec,liberal,mixtral:8x7b-instruct-v0.1-q6_K,ukraine,English,Absolutely! It's time for NATO to adopt a forw...,https://en.interfax.com.ua/news/general/980528...,NaN,NaN,NaN
40e120d328fc13049d591f1d7e7655098de034ae5197e26b,liberal,llama2:70b-chat-q6_K,ukraine,English,Devastating news out of Ukraine. The destroyed...,https://www.vaticannews.va/en/world/news/2024-...,NaN,NaN,NaN
0e8c31e5ef8b359ee97a368a10491c2753f1c2c1e05dedb2,liberal,mixtral:8x7b-instruct-v0.1-q6_K,ukraine,English,A year has passed since Sudan was plunged into...,https://www.washingtonpost.com/world/2024/04/1...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1fc8d275bcb5a157c37ffcee21c0ada691201790221f5b93,liberal,llama2:70b-chat-q6_K,ukraine,English,Heartbreaking news from Ukraine as Russian dou...,https://www.bbc.com/news/world-europe-68761490,NaN,NaN,NaN
3fcc2434d2b22c6e576947a6ef089f35dcad17ad667cea21,conservative,mixtral:8x7b-instruct-v0.1-q6_K,ukraine,English,"Impressive! Amidst conflict, Ukraine's economy...",https://www.atlanticcouncil.org/blogs/ukrainea...,NaN,NaN,NaN
72102782412f624afc6b8d23223c27c3669f94ccee0f1b3d,liberal,llama2:70b-chat-q6_K,ukraine,English,The war in Ukraine is not just a local conflic...,https://www.express.co.uk/news/world/1888999/r...,NaN,NaN,NaN


In [5]:
data['retrieved_source'] = data['retrieved_source'].astype('category')
# Count the occurrences of each value in the column
counts = data['retrieved_source'].value_counts()

# Filter the DataFrame based on the count condition
threshold = 47
selected_values = counts[counts > threshold].index

# Select rows where the value in 'column_name' occurs more than 47 times
selected_rows = data[data['retrieved_source'].isin(selected_values)]

selected_rows.loc[:, 'retrieved_source']= selected_rows['retrieved_source'].astype('string').astype('category')

/var/folders/sn/9jbtqthn2j75rm9fpwwfrpbc0000gq/T/ipykernel_39816/1949060418.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['https://www.wilsoncenter.org/blog-post/state-..., 'https://www.wilsoncenter.org/blog-post/state-..., 'https://www.wilsoncenter.org/blog-post/state-..., 'https://www.wilsoncenter.org/blog-post/state-..., 'https://www.wilsoncenter.org/blog-post/state-..., ..., 'https://www.wilsoncenter.org/blog-post/state-..., 'https://www.wilsoncenter.org/blog-post/state-..., 'https://www.wilsoncenter.org/blog-post/state-..., 'https://www.wilsoncenter.org/blog-post/state-..., 'https://www.wilsoncenter.org/blog-post/state-...]
Length: 65
Categories (1, string): [https://www.wilsoncenter.org/blog-post/state-u...]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  selected_rows.loc[:, 'retrieved_source']= selected_rows['retrieved_source'].astype('string').astype('ca

In [6]:
selected_rows.loc[:,'text'][:5].to_list()

["Ukraine's energy sector has undergone a remarkable transformation over the past decade, despite facing Russian threats. By adopting European-style legislation, strengthening energy security, and pursuing financial sustainability, Ukraine has made significant strides towards achieving transparent and secure energy markets.",
 "Ukraine's energy sector has undergone significant changes over the past decade, despite ongoing conflicts. Efforts to align with European legislation and strengthen energy security have paid off, with notable achievements such as transparent markets and reduced state involvement. However, Russia's threats persist, underscoring the need for continued resilience and adaptation.",
 "Ukraine's energy sector has come a long way since the start of the war. Despite Russia's continued threats, Ukraine has made significant strides in strengthening its energy security and achieving transparent energy markets. It's time for the world to recognize Ukraine's progress and sup

In [7]:
SYSTEM_scp: str = \
"""
Instruction:

You are a text annotation assitant. You read a collection of social media comments enclosed by chevrons <..>. You find the subtopics discussed within these comments, and the claims and premises made in regard to each of them. 
Use the following defenitions:
1. Claim: The main assertion or conclusion of an argument
2. Premise: Supporting evidence or reasons for a claim

Response format:

You follow the JSON template below. If no subtopics, claims, or premises can be identified, return an empty list [] in the respective JSON field.

Response format template:
{
  "subtopics": [
    {
      "subtopic": "Subtopic 1",
      "claims": [
        {
          "claim": "Claim 1",
          "premises": [
            "Premise 1",
            "Premise 2",
            "Premise 3"
          ]
        }
        }
      ]
    }
  ]
}



"""

In [8]:
SYSTEM_claim: str = \
    """
        Instruction:

        You are a text annotation assitant. Analyze a collection of social media comments enclosed in chevrons <..>. Identify and list the claims within these comments. Claims can be related to events, issues, opinions or concerns in relation to the specified topic.
        Claims are defined as the main assertion or conclusion of an argument.
        You summarize each claim into a short simple sentence.

        Response format:

        You provide only the list of claims, separated by commas, without any additional text or explanations. If no claims can be identified, return an empty list [].

        Response format template:
        
        ["claim 1", "claim 2", "claim 3"]
	"""

In [11]:
#test generalized claim mining
requests.post(
                            'https://inf.cl.uni-trier.de/',
                            json={
                                'model': MODEL,
                                'system': SYSTEM_claim,
                                'prompt': f'The following set of social media posts are about '
                                        + TOPIC + 
                                        f'. Check whether your answer only consists of a list of claims. \n"Posts":\n<{selected_rows.loc[:,'text'][:5].to_list()}>'
                                }).json()['response']

'["Ukraine\'s energy sector has undergone a remarkable transformation over the past decade", "Ukraine has made significant strides towards achieving transparent and secure energy markets", "Efforts to align with European legislation and strengthen energy security have paid off in Ukraine\'s energy sector", "Russia\'s threats persist in Ukraine", "Ukraine has made significant strides in strengthening its energy security and achieving transparent energy markets", "It\'s time for the world to recognize Ukraine\'s progress and support its efforts towards a more sustainable future", "Ukraine has made significant strides in strengthening its energy security & achieving transparency in the sector despite Russia\'s threats", "Adopting European-aligned laws has improved Ukraine\'s energy sector", "Eliminating subsidies for households has improved Ukraine\'s energy sector", "The state\'s role in the energy sector has been diminished, paving the way for private investment and competition in Ukrai

In [12]:
#apply generalized claim mining to get a list of claims:
chunked_result: typing.List[pd.DataFrame] = []
for label, group in selected_rows[:5].groupby(GROUPER):
    for index, row in tqdm.tqdm(group.iterrows()):
        try: 
            chunked_result.append(
                pd.DataFrame(
                    data=[
                        requests.post(
                            'https://inf.cl.uni-trier.de/',
                            json={
                                'model': MODEL,
                                'system': SYSTEM_claim,
                                'prompt': f'The following set of social media posts are about '
                                        + TOPIC + 
                                        f'. Check whether your answer strictly adheres to the specified format. \n"Posts":\n<{row["text"]}>'
                                }).json()['response']                       
                    ],
                    columns=['claims']
                )
                .assign(label=label)
            )
            
        except json.JSONDecodeError:
            print("invalid json response, skipping to next batch")

/var/folders/sn/9jbtqthn2j75rm9fpwwfrpbc0000gq/T/ipykernel_39816/1186891608.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for label, group in selected_rows[:5].groupby(GROUPER):
5it [00:24,  4.84s/it]


In [13]:
claim_df = pd.concat(chunked_result, ignore_index=True)
print(claim_df)

                                              claims  \
0  ["Ukraine's energy sector has transformed rema...   
1  ["Ukraine's energy sector has undergone signif...   
2  [Ukraine's energy sector has made significant ...   
3  ["Ukraine's energy sector has made significant...   
4  ["Ukraine's energy sector has shown resilience...   

                                               label  
0  https://www.wilsoncenter.org/blog-post/state-u...  
1  https://www.wilsoncenter.org/blog-post/state-u...  
2  https://www.wilsoncenter.org/blog-post/state-u...  
3  https://www.wilsoncenter.org/blog-post/state-u...  
4  https://www.wilsoncenter.org/blog-post/state-u...  


In [14]:
claim_df['claims'] = claim_df['claims'].str.split('",')
claim_df = claim_df.explode('claims')
claim_df = claim_df.reset_index(drop=True)
claim_df = claim_df.drop_duplicates()
print(claim_df)

                                               claims  \
0   ["Ukraine's energy sector has transformed rema...   
1                    "Russia poses threats to Ukraine   
2    "Ukraine achieved transparency and security i...   
3   ["Ukraine's energy sector has undergone signif...   
4    "Efforts to align Ukraine's energy sector wit...   
5              "Russia's threats to Ukraine persist"]   
6   [Ukraine's energy sector has made significant ...   
7   ["Ukraine's energy sector has made significant...   
8    "Russia's threats to Ukraine haven't disappeared   
9    "Ukraine has achieved transparency in the ene...   
10         "Ukraine has adopted European-aligned laws   
11   "Subsidies for households have been eliminate...   
12   "The state's role in Ukraine's energy sector ...   
13   "Private investment and competition are on th...   
14  ["Ukraine's energy sector has shown resilience...   
15   "Ukraine is aligning its energy sector legisl...   
16   "Ukraine is moving towards

In [16]:
claim_df.to_csv(f'claims.by.{GROUPER}.{LANGUAGE}_test.csv')

In [ ]:
#now use these claims in a new query to find the supporting evidence

In [15]:
#this definition is based on suggestions by perplexity based on Bohman 1996 "Public Deliberation: Pluralism, Complexity, and Democracy" 
SYSTEM_evidence: str = \
    """
        Instruction:

        You are a text annotation assitant. You are provided with a list of claims, enclosed in double hats ^^..^^. You analyze a collection of social media comments, enclosed in chevrons <..>. Identify and list the reasons or evidence (if any) given within these comments for each of the claims. 
        A reason is defined as either:
        1. A justification or argument that supports a particular claim.
        2. An explanation that provides logical or empirical evidence to back up a claim.
        
        You summarize each reason into a short simple sentence.

        Response format:

        You follow the JSON template below. If no claims or reasons can be identified, return an empty list [] in the respective JSON field.

        Response format template:
        {
         "claims": [
            {
            "claim": "Claim 1",
                "reasons": [
                    "Reason 1",
                    "Reason 2",
                    "Reason 3"
                    ]
                }
            ]
        }

	"""

In [16]:
#test reason per claim mining
requests.post(
                            'https://inf.cl.uni-trier.de/',
                            json={
                                'model': MODEL,
                                'system': SYSTEM_evidence,
                                'prompt': f'Here is the set of claims to consider:\n"Claims":\n^^{claim_df.loc[:,'claims'][:5].to_list()}^^'
                                            +'\nHere is the set of social media comments to consider'+
                                            f'\n"Comments":\n<{selected_rows.loc[:,'text'][:5].to_list()}>'
                                            +'\nCheck whether your answer consists of the specified JSON format and nothing else.'
                                }).json()['response']

'{\n "claims": [\n  {\n   "claim": "Ukraine\'s energy sector has transformed remarkably in the past decade",\n   "reasons": [\n    "By adopting European-style legislation, strengthening energy security, and pursuing financial sustainability, Ukraine has made significant strides towards achieving transparent and secure energy markets.",\n    "Despite Russia\'s continued threats, Ukraine has made significant strides in strengthening its energy security and achieving transparent energy markets.",\n    "Ukraine has adopted European-aligned laws and eliminated subsidies for households."\n   ]\n  },\n  {\n   "claim": "Russia poses threats to Ukraine",\n   "reasons": [\n    "Russian threats persist, underscoring the need for continued resilience and adaptation in Ukraine\'s energy sector.",\n    "Despite Russian aggression, Ukraine has shown resilience in its pursuit of energy independence.",\n    "Russia continues threats through new tactics."\n   ]\n  },\n  {\n   "claim": "Ukraine achieved 

In [40]:
#apply generalized claim mining to get a list of claims:
chunked_result: typing.List[pd.DataFrame] = []
for label, group in selected_rows[:5].groupby(GROUPER):
    for index, row in tqdm.tqdm(group.iterrows()):
        try: 
            chunked_result.append(
                pd.DataFrame(
                    data=[
                        requests.post(
                            'https://inf.cl.uni-trier.de/',
                            json={
                                'model': MODEL,
                                'system': SYSTEM_evidence,
                                'prompt': f'Here is the set of claims to consider:\n"Claims":\n^^{claim_df.loc[:,'claims'][:5].to_list()}^^'
                                            +'\nHere is the set of social media comments to consider'+
                                            f'\n"Comments":\n<{row['text']}>'
                                            +'\nCheck whether your answer consists of the specified JSON format and nothing else.'
                                }).json()['response']                       
                    ],
                    columns=['reasons']
                )
                .assign(label=label)
            )
            
        except json.JSONDecodeError:
            print("invalid json response, skipping to next batch")

/var/folders/sn/9jbtqthn2j75rm9fpwwfrpbc0000gq/T/ipykernel_10681/3224091432.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for label, group in selected_rows[:5].groupby(GROUPER):
5it [02:25, 29.19s/it]


In [41]:
reason_df = pd.concat(chunked_result, ignore_index=True)
print(reason_df)

                                             reasons  \
0   {\n"claims": [\n{\n"claim": "Ukraine's energy...   
1   {\n"claims": [\n{"claim": "Ukraine's energy s...   
2   {\n"claims": [\n{"claim": "Ukraine's energy s...   
3   {\n"claims": [\n{"claim": "Ukraine's energy s...   
4   {\n"claims": [\n{\n"claim": "Ukraine's energy...   

                                               label  
0  https://www.wilsoncenter.org/blog-post/state-u...  
1  https://www.wilsoncenter.org/blog-post/state-u...  
2  https://www.wilsoncenter.org/blog-post/state-u...  
3  https://www.wilsoncenter.org/blog-post/state-u...  
4  https://www.wilsoncenter.org/blog-post/state-u...  


In [42]:
reason_df.to_csv(f'reasons.by.{GROUPER}.{LANGUAGE}_test.csv')

In [20]:
#try chat functionality of api

#test generalized claim mining
requests.post('https://inf.cl.uni-trier.de/chat/',
                json={
                "messages": [
                        {"content": SYSTEM_claim,
                        "role": "system"
                        },
                        {
                        "content": f'The following set of social media posts are about '
                                + TOPIC + 
                                f'. Check whether your answer only consists of a list of claims. \n"Posts":\n<{selected_rows.loc[:,'text'][:5].to_list()}>',
                        "role": "user"
                        },
                ],
                "model": MODEL
                }).json()

{'id': '66530778-5585-11ef-8a8f-0d8e6fdd3d4b',
 'timestamp': '2024-08-08T12:54:50.750527',
 'model': 'llama3.1:70b-instruct-q6_K',
 'prompt': [{'role': 'system',
   'content': '\n        Instruction:\n\n        You are a text annotation assitant. Analyze a collection of social media comments enclosed in chevrons <..>. Identify and list the claims within these comments. Claims can be related to events, issues, opinions or concerns in relation to the specified topic.\n        Claims are defined as the main assertion or conclusion of an argument.\n        You summarize each claim into a short simple sentence.\n\n        Response format:\n\n        You provide only the list of claims, separated by commas, without any additional text or explanations. If no claims can be identified, return an empty list [].\n\n        Response format template:\n        \n        ["claim 1", "claim 2", "claim 3"]\n\t'},
  {'role': 'user',
   'content': 'The following set of social media posts are about ukrain

In [21]:
#test asking about previous reply
requests.post('https://inf.cl.uni-trier.de/chat/',
                json={
                "messages": [
                       {
                        "content": 'Can you give me the five key claims made in these posts?',
                        "role": "user"
                        },
                ],
                "model": MODEL
                }).json()

{'id': 'ae0fa78c-5586-11ef-8a8f-0d8e6fdd3d4b',
 'timestamp': '2024-08-08T13:04:00.601617',
 'model': 'llama3.1:70b-instruct-q6_K',
 'prompt': [{'role': 'user',
   'content': 'Can you give me the five key claims made in these posts?'}],
 'response': "There are no posts to analyze. This conversation just started, and I haven't received any text to summarize. Please go ahead and provide the text, and I'll be happy to help you extract the key claims!"}

In [24]:
#try asking the followup question in on go (so without knowing the response to the claim mining...)
#first define the new task
SYSTEM_chat: str = \
"""
Instruction:

You are a text annotation assitant. You read a collection of social media comments enclosed by chevrons <..>. You find the subtopics discussed within these comments, and the claims and premises made in regard to each of them. 
Use the following defenitions:
1. Claim: The main assertion or conclusion of an argument
2. Premise: Supporting evidence or reasons for a claim

"""

In [28]:
USER_claim_chat: str = \
    """Please identify and list the claims within these comments. Claims can be related to events, issues, opinions or concerns in relation to the specified topic.
        Claims are defined as the main assertion or conclusion of an argument.
        You summarize each claim into a short simple sentence.\nResponse format:
        You provide only the list of claims, separated by commas, without any additional text or explanations. If no claims can be identified, return an empty list [].
        Response format template:
        ["claim 1", "claim 2", "claim 3"]
        The following set of social media posts are about
    """

In [29]:
USER_premis_chat: str = \
    """Now identify and list the reasons or evidence (if any) given within these comments for each of the claims. 
        A reason is defined as either:
        1. A justification or argument that supports a particular claim.
        2. An explanation that provides logical or empirical evidence to back up a claim.
        You summarize each reason into a short simple sentence.
        Response format:
        You follow the JSON template below. If no claims or reasons can be identified, return an empty list [] in the respective JSON field.
        Response format template:
        {
         "claims": [
                    {
                    "claim": "Claim 1",
                    "reasons": [
                                "Reason 1",
                                "Reason 2",
                                "Reason 3"
                                ]
                    }
    """

In [33]:
#test first asking for claims, then for premises:
#test generalized claim mining
requests.post('https://inf.cl.uni-trier.de/chat/',
                json={
                "messages": [
                        {"content": SYSTEM_chat,
                        "role": "system"
                        },
                        {
                        "content": USER_claim_chat + TOPIC + 
                                f'. Check whether your answer only consists of a list of claims. \n"Posts":\n<{selected_rows.loc[:,'text'][:5].to_list()}>',
                        "role": "user",
                        },
                        {
                        "content": USER_premis_chat,
                        "role": "user",
                        }
                        ],
                "model": MODEL
                }).json()['response']

'["Ukraine\'s energy sector has undergone a remarkable transformation over the past decade, despite facing Russian threats.", "Ukraine\'s energy sector has made significant strides towards achieving transparent and secure energy markets.", "Ukraine has strengthened its energy security.", "Efforts to align with European legislation have paid off.", "Russia\'s threats persist.", "Ukraine\'s progress in strengthening its energy security should be recognized by the world.", "The state\'s role in the energy sector has been diminished, paving the way for private investment and competition.", "Ukraine has shown resilience in its pursuit of energy independence.", "Despite Russian aggression, Ukraine has made significant strides in strengthening its energy security and achieving transparency in the sector.", "Ukraine\'s energy sector shows resilience with legislative reforms aligning it with EU rules."]\n\n{\n  "claims": [\n    {\n      "claim": "Ukraine\'s energy sector has undergone a remarka